In [1]:
import vk 
import json

In [2]:
token = "b2abdafff949da3a6c493c8b4c8888ad62ff567c0c9953a7267eb677e2b4f450212e5c18dc878367522af"  # Сервисный ключ доступа, по завершении дз можно удалить для соблюдения конфиденциальности
session = vk.Session(access_token=token)  # Авторизация
vk_api = vk.API(session)  # Авторизация

In [3]:
personal_token = "69bc212568d9bea2e42f1edab85d546f1785ca776800441ce3db12ed4c5b3b7d87fc9240cb2635937d2e1"  # Персональный ключ доступа, по завершении дз можно удалить для соблюдения конфиденциальности
session = vk.Session(access_token=personal_token)  # Авторизация
vk_personal_api = vk.API(session)  # Авторизация

In [4]:
def get_members(groupid, fields=('sex', 'bdate', 'city', 'country', 'domain')):
    first = vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92)  # Первое выполнение метода
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 1000  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data = data + vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92, offset=i*1000)["items"]
    return data

In [5]:
members= get_members ('trmuzik')

VkAPIError: 6. Too many requests per second. request_params = {'method': 'groups.getMembers', 'oauth': '1', 'group_id': 'trmuzik', 'fields': 'sex,bdate,city,country,domain', 'v': '5.92', 'offset': '17000'}

In [ ]:
len (members)

In [32]:
members_ts1 = get_members ('hercaiturksinema')

In [33]:
len (members_ts1)

306496

In [27]:
members_ts2 = get_members ('dizimania')

In [28]:
len (members_ts2)

156650

In [43]:
members_ts3 = get_members ('diziturk')

In [44]:
len (members_ts3)

199077

In [59]:
def save_members_to_file(filename, members):
    with open(filename, 'w', encoding='utf-8') as f:
        for member in members:
            _id = member["id"] # находим айди участника
            first_name = member["first_name"] # находим имя учатника
            last_name = member["last_name"] # находим фамилию учатника
            
            sex = member["sex"] # находим пол учатника (1 или 2)
            if int(sex) == 1:  # если пол это 1 
                sex = "F"  # обозначем женский пол как F
            elif int(sex) == 2:  # если пол это 2 
                sex = "M" # обозначем мужской пол как F
            else:  # если есть другой какой-то пол
                sex = "-"  # скорее всего нет, поэтому ставим прочерк
                
            bdate = member.get('bdate', "-")   # находим день рождения учатника
            if len(bdate.split(".")) == 3:  # нам нужен только год, если он есть
                bdate = bdate.split(".")[-1]  # то сохраняем год
            else:  # если год не указан, то возраст участника мы не узнаем
                bdate = "-"  # поэтому прочерк
                
            city = member.get('city', {}).get("title", "-")  # находим город участника
            country = member.get('country', {}).get("title", "-")  # находим страну участника

In [62]:
save_members_to_file ('Turk_music.tsv', members)

In [64]:
save_members_to_file ('hercaiturksinema.tsv', members_ts1)

TypeError: string indices must be integers

In [65]:
save_members_to_file ('dizimania.tsv', members_ts2)

TypeError: string indices must be integers

In [49]:
save_members_to_file ('diziturk.tsv', members_ts3)

In [ ]:
import pandas as pd

In [33]:
members_ts1 = pd.read_csv('hercaiturksinema.tsv', sep = '\t', 
            names = ['id', 'first_name', 'last_name', 'sex', 'bdate', 'city', 'country'])

members_ts2 = pd.read_csv('dizimania1.tsv', sep = '\t', 
            names = ['id', 'first_name', 'last_name', 'sex', 'bdate', 'city', 'country'])

members_ts3 = pd.read_csv('diziturk.tsv', sep = '\t', 
            names = ['id', 'first_name', 'last_name', 'sex', 'bdate', 'city', 'country'])

In [34]:
members_ts3 = set(members_ts3)
members_ts1 = set(members_ts1)
members_ts2 = set(members_ts2)

In [15]:
united_members = members_ts1.union(members_ts2).union(members_ts3)

In [28]:
len(united_members)

595537

In [16]:
intersected_members = members_ts1.intersection(members_ts2).intersection(members_ts3)

In [1]:
set([1, 2, 3]) - set([2, 3])

{1}

In [31]:
#для первой группы берем всех пользователей,потому что они все уникальные
members_ts1_unq = members_ts1
#для второй группы считаем пользователей, которые есть во второй группе, но нет в первой
members_ts2_unq = members_ts2 - members_ts1
#для третьей группы считаем пользователей, которые есть во третьей группе, но нет в первой и второй
members_ts3_unq = members_ts3 - members_ts2 - members_ts1

In [35]:
#для подсчета пользователей, которые смотрят сериалы и слушают музыку
united_members = members_ts1 & members_ts2

In [23]:
len(intersected_members)

7440

In [26]:
pd.DataFrame(intersected_members, columns = ['id']).to_csv('intersection.tsv', sep = '\t', index = False)

In [27]:
pd.read_csv('intersection.tsv', sep = '\t')

,id
0,"269033732,"
1,"257728828,"
2,"138509261,"
3,"44125893,"
4,"8773559,"
...,...
7435,"158834960,"
7436,"11209825,"
7437,"471922168,"
7438,"373673668,"
